In [1]:
%%capture
from google.colab import drive
drive.mount('/content/drive')

import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm datasets
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm datasets
# Install latest Hugging Face for Gemma-3!
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

In [2]:
!pip install bitsandbytes
!pip install unsloth_zoo
!pip install msgspec
!pip install blake3
!pip install gguf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 4.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vllm 0.8.2 requires blake3, which is not installed.
vllm 0.8.2 requires compressed-tensors==0.9.2, which is not installed.
vllm 0.8.2 requires depyf==0.18.0, which is not installed.
vllm 0.8.2 requires fastapi[standard]>=0.115.0, which is not installed.
vllm 0.8.2 requires gguf==0.10.0, which is not installed.
vllm 0.8.2 requires lark==1.2.2, which is not installed.
vllm 0.8.2 requires llguidance<0.8.0,>=0.7.9; platform_machine == "x86_64" or platform_machine == "arm64" or platform_machine == "aarch64", which is not installed.
vllm 0.8.2 requires lm-format-enforcer<0.11,>=0.10.11, which is not installed.
vllm 0.8.2 requires mistral_common[opencv]>=1.5.4, which is not installed.
vllm 0.8.2 requires ninja, which is not installed.
vllm 0.

# Load up Gemma 3 4B Instruct, and set parameters

In [3]:
from unsloth import FastModel
import torch
max_seq_length = 4096

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 03-31 18:02:17 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/192 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

# We now add LoRA adapters so we only need to update a small amount of parameters!

In [4]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 16,           # Larger = higher accuracy, but might overfit
    lora_alpha = 32,  # Recommended alpha == r at least
    lora_dropout = 0.05,
    bias = "none",
    random_state = 2,
)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


# Data Prep

We're using the FinQA dataset that's been prepared for reasoning tasks

In [5]:
import json
import re
from datasets import Dataset

def load_finqa_dataset(file_path, system_prompt):
    """
    Load the FinQA dataset from a JSONL file and format it for GRPO training

    Args:
        file_path: Path to the JSONL file
        system_prompt: System prompt to use for each example

    Returns:
        Dataset: A HuggingFace dataset containing the formatted data
    """
    data = []

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            item = json.loads(line)

            # Format the item into the expected structure
            formatted_item = {
                "prompt": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": item["question"]},
                ],
                "answer": item["answer"],
                "response": item["response"],
                "extracted": item["extracted"]
            }

            data.append(formatted_item)

    # Convert to HuggingFace dataset
    return Dataset.from_list(data)

In [6]:
import os
os.getcwd()

'/content'

In [7]:
# Define the format tags
reasoning_start = "<start_working_out>"
reasoning_end   = "<end_working_out>"
solution_start = "<SOLUTION>"
solution_end = "</SOLUTION>"

system_prompt = f"""You are given a financial problem.
Think about the problem and provide your working out.
Place it between {reasoning_start} and {reasoning_end}.
Then, provide your solution between {solution_start}{solution_end}"""

# Change these paths if needed based on your setup
train_path = "/content/drive/MyDrive/2025_ConvFinQA_SFT_Agentic/data/processed_datasets/finqa_reasoning_train.jsonl"
dev_path = "/content/drive/MyDrive/2025_ConvFinQA_SFT_Agentic/data/processed_datasets/finqa_reasoning_dev.jsonl"

# Load datasets
train_dataset = load_finqa_dataset(train_path, system_prompt)
dev_dataset = load_finqa_dataset(dev_path, system_prompt)

print(f"Loaded {len(train_dataset)} training examples")
print(f"Loaded {len(dev_dataset)} dev examples")

Loaded 3965 training examples
Loaded 542 dev examples


In [8]:
# Let's look at the first row
train_dataset[0]["prompt"][1]["content"][:200] # First 200 chars of the first question

'26 | 2009 annual report in fiscal 2008 , revenues in the credit union systems and services business segment increased 14% ( 14 % ) from fiscal 2007 . all revenue components within the segment experien'

In [9]:
train_dataset[0]["answer"] # The answer for the first example

'14.1%'

# We create a regex format to match the reasoning sections and answers:

In [10]:
import re

match_format = re.compile(
    rf"^[\s]{{0,}}"\
    rf"{reasoning_start}.+?{reasoning_end}.*?"\
    rf"{solution_start}(.+?){solution_end}"\
    rf"[\s]{{0,}}$",  # optional whitespace at end
    flags = re.MULTILINE | re.DOTALL
)

In [11]:
# We verify it works
match_format.search(
    "<start_working_out>Let me think!<end_working_out>"\
    "<SOLUTION>142.4%</SOLUTION>",  # solution
)

<re.Match object; span=(0, 76), match='<start_working_out>Let me think!<end_working_out>>

# We now want to create a reward function to match the format exactly - we reward it with 3 points if it succeeds:

In [12]:
def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Match if format is seen exactly!
        if match_format.search(response) is not None: score += 3.0
        scores.append(score)
    return scores

# If it fails, we want to reward the model if it at least follows the format partially, by counting each symbol:

In [13]:
def match_format_approximately(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Count how many keywords are seen - we penalize if too many!
        # If we see 1, then plus some points!
        score += 0.5 if response.count(reasoning_start) == 1 else -0.5
        score += 0.5 if response.count(reasoning_end)   == 1 else -0.5
        score += 0.5 if response.count(solution_start)  == 1 else -0.5
        score += 0.5 if response.count(solution_end)    == 1 else -0.5
        scores.append(score)
    return scores

# Finally, we want to extract the generated answer, and reward or penalize it! We also reward it based on how close the answer is to the true one via ratios:

In [14]:
def check_answer(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_format.search(r)) is not None else None   # extract answer from solution tags
        for r in responses
        ]

    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        score = 0
        if guess is None:
            scores.append(0)
            continue

        # Correct answer gets 3 points!
        if guess == true_answer:
            score += 3.0
        # Match if spaces are seen
        elif guess.strip() == true_answer.strip():
            score += 1.5
        else:
            # Handle percentage values
            try:
                # Try to normalize percentages
                guess_clean = guess.strip().replace('%', '')
                true_answer_clean = true_answer.strip().replace('%', '')

                ratio = float(guess_clean) / float(true_answer_clean)
                if   ratio >= 0.9 and ratio <= 1.1: score += 0.5
                elif ratio >= 0.8 and ratio <= 1.2: score += 0.25
                else: score -= 1.0 # Penalize wrong answers
            except:
                score -= 0.5 # Penalize
        scores.append(score)
    return scores

# Also sometimes it might not be 1 number as the answer, but like a sentence for example "The solution is $20" -> we extract 20.

In [15]:
match_numbers = re.compile(
    rf"{solution_start}.*?([\d\.]{{1,}})",
    flags = re.MULTILINE | re.DOTALL
)
match_numbers.findall("<SOLUTION>  142.4%  </SOLUTION>")

['142.4']

In [16]:
def check_numbers(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_numbers.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    # Print debug info for first item
    if len(extracted_responses) > 0 and len(answer) > 0:
        print('*'*20, f"Question:\n{question[:100]}...",
              f"\nAnswer:\n{answer[0]}",
              f"\nResponse:\n{responses[0][:100]}...",
              f"\nExtracted:\n{extracted_responses[0]}")

    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(0)
            continue
        # Convert to numbers
        try:
            # Try to normalize percentages
            guess_clean = guess.strip().replace('%', '')
            true_answer_clean = true_answer.strip().replace('%', '')

            true_answer_num = float(true_answer_clean)
            guess_num = float(guess_clean)
            scores.append(1.5 if guess_num == true_answer_num else 0.0)
        except:
            scores.append(0)
            continue
    return scores

# Let's add one more reward function to encourage detailed reasoning

In [17]:
def reasoning_quality_reward_func(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Look for the reasoning section
        reasoning_match = re.search(
            rf"{reasoning_start}(.*?){reasoning_end}",
            response,
            re.DOTALL
        )
        if reasoning_match:
            reasoning = reasoning_match.group(1)
            # Count calculation steps (lines with = or mathematical symbols)
            calculation_steps = len(re.findall(r'[\d\s]+[+\-*/][\d\s]+=', reasoning))
            # Reward based on number of steps (more steps = more detailed reasoning)
            score += min(calculation_steps, 3) * 0.5
        scores.append(score)
    return scores

# Train the model
Now set up GRPO Trainer and all configurations!

In [23]:
max_prompt_length = 2048 + 1024

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_torch_fused",
    logging_steps = 10,
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 200, # Increase for better results
    save_steps = 50,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "/content/drive/MyDrive/2025_ConvFinQA_SFT_Agentic/output_grpo/gemma-3-4b-finqa",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 2 to the `num_generations` of 4


# And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the reward column increase!
You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!
Step	Training Loss	reward	reward_std	completion_length	kl
1	0.000000	0.125000	0.000000	200.000000	0.000000
2	0.000000	0.072375	0.248112	200.000000	0.000000
3	0.000000	-0.079000	0.163776	182.500000	0.000005

In [24]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        match_format_exactly,
        match_format_approximately,
        check_answer,
        check_numbers,
        reasoning_quality_reward_func
    ],
    args = training_args,
    train_dataset = train_dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,965 | Num Epochs = 1 | Total steps = 200
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 29,802,496/4,000,000,000 (0.75% trained)


******************** Question:
there is no goodwill assigned to reporting units within the balance sheet management segment . the f... 
Answer:
7% 
Response:
<start_working_out>
Total goodwill at December 31, 2012, is $1,934.2 million (as stated in the repor... 
Extracted:
7.42
******************** Question:
112 / sl green realty corp . 2017 annual report 20 . commitments and contingencies legal proceedings... 
Answer:
23.1% 
Response:
<start_working_out>
The question asks for the percentage of the total plan contributions in 2017 tha... 
Extracted:
23.06
******************** Question:
nike , inc . notes to consolidated financial statements 2014 ( continued ) such agreements in place ... 
Answer:
91% 
Response:
<start_working_out>

The question is: what was the percentage gain on the sale of starter brand busi... 
Extracted:
47.67
******************** Question:
kimco realty corporation and subsidiaries notes to consolidated financial statements , continued the... 
Answer:
4.8 
Response

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / match_format_exactly,rewards / match_format_approximately,rewards / check_answer,rewards / check_numbers,rewards / reasoning_quality_reward_func
10,0.000000,2.753125,1.710900,270.743750,0.001128,1.293750,1.268750,-0.112500,0.121875,0.181250
20,0.000000,3.059375,1.724954,277.687500,0.001071,1.331250,1.337500,0.012500,0.215625,0.162500
30,0.034800,3.253125,1.726038,287.356250,0.869466,1.481250,1.418750,0.084375,0.159375,0.109375
40,0.000100,3.600000,1.819778,307.768750,0.001487,1.875000,1.500000,-0.075000,0.196875,0.103125
50,0.000100,3.546875,1.855536,271.900000,0.002852,1.743750,1.475000,-0.068750,0.281250,0.115625
60,0.000200,4.545312,1.547251,240.787500,0.004647,2.400000,1.731250,0.079687,0.140625,0.193750
70,0.000300,4.789062,1.212910,281.143750,0.007065,2.643750,1.750000,-0.023438,0.215625,0.203125
80,0.000300,4.206250,1.209908,288.918750,0.007395,2.493750,1.693750,-0.271875,0.121875,0.168750
90,0.000300,4.685937,1.057762,268.950000,0.007502,2.662500,1.875000,-0.095312,0.037500,0.206250


******************** Question:
jpmorgan chase & co./2009 annual report 181 the following table shows the current credit risk of der... 
Answer:
80968000000 
Response:
<start_working_out>
The total collateral of all types as of December 31, 2009 is calculated as follo... 
Extracted:
44.1
******************** Question:
management 2019s discussion and analysis results of reportable business segments net sales segment i... 
Answer:
43900000 
Response:
<start_working_out>
Sales increased 2% ( 2 % ) due to higher selling prices.
-----------------------... 
Extracted:
2
******************** Question:
part a0iii item a010 . directors , executive officers and corporate governance for the information r... 
Answer:
70.9% 
Response:
<start_working_out>
The question is: what portion of the securities approved by the security holders... 
Extracted:
3578241
******************** Question:
hologic , inc . notes to consolidated financial statements ( continued ) ( in thousands , except per... 
Answer:
1

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / match_format_exactly,rewards / match_format_approximately,rewards / check_answer,rewards / check_numbers,rewards / reasoning_quality_reward_func
10,0.000000,2.753125,1.710900,270.743750,0.001128,1.293750,1.268750,-0.112500,0.121875,0.181250
20,0.000000,3.059375,1.724954,277.687500,0.001071,1.331250,1.337500,0.012500,0.215625,0.162500
30,0.034800,3.253125,1.726038,287.356250,0.869466,1.481250,1.418750,0.084375,0.159375,0.109375
40,0.000100,3.600000,1.819778,307.768750,0.001487,1.875000,1.500000,-0.075000,0.196875,0.103125
50,0.000100,3.546875,1.855536,271.900000,0.002852,1.743750,1.475000,-0.068750,0.281250,0.115625
60,0.000200,4.545312,1.547251,240.787500,0.004647,2.400000,1.731250,0.079687,0.140625,0.193750
70,0.000300,4.789062,1.212910,281.143750,0.007065,2.643750,1.750000,-0.023438,0.215625,0.203125
80,0.000300,4.206250,1.209908,288.918750,0.007395,2.493750,1.693750,-0.271875,0.121875,0.168750
90,0.000300,4.685937,1.057762,268.950000,0.007502,2.662500,1.875000,-0.095312,0.037500,0.206250
100,0.000300,4.582813,1.395074,279.925000,0.008062,2.531250,1.637500,-0.064062,0.206250,0.271875


******************** Question:
leases , was $ 92 million , $ 80 million , and $ 72 million in 2002 , 2001 , and 2000 , respectively... 
Answer:
17% 
Response:
Here's the breakdown and calculation:

*   **Total Minimum Lease Payments:** $464 (as stated in the ... 
Extracted:
None
******************** Question:
14 . leases we lease certain locomotives , freight cars , and other property . the consolidated stat... 
Answer:
24% 
Response:
<start_working_out>
The problem asks for the percentage of total minimum lease payments that are cap... 
Extracted:
32.1
******************** Question:
entergy corporation and subsidiaries management 2019s financial discussion and analysis combination ... 
Answer:
-7.4% 
Response:
<start_working_out>
The growth rate in net revenue is calculated as follows:
*   Net Revenue in 2015... 
Extracted:
7.5
******************** Question:
note 9 2014goodwill and other intangibles , net goodwill the following table outlines the activity i... 
Answer:
-7% 
Response:


KeyboardInterrupt: 

# Inference
Now let's try the model we just trained!

In [25]:
# Choose a financial question from our dataset or create a new one
test_question = "If a company's revenue increased from $1.2 million to $1.5 million, what is the percentage increase?"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": test_question},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 256, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<start_working_out>
To calculate the percentage increase, we need to find the amount of increase and then divide it by the original amount, and finally multiply by 100.

1.  **Calculate the amount of increase:**
    Increase = New Revenue - Original Revenue
    Increase = $1.5 million - $1.2 million
    Increase = $0.3 million

2.  **Calculate the percentage increase:**
    Percentage Increase = (Increase / Original Revenue) * 100
    Percentage Increase = ($0.3 million / $1.2 million) * 100
    Percentage Increase = 0.25 * 100
    Percentage Increase = 25%

Therefore, the percentage increase in revenue is 25%.
</end_working_out>

<SOLUTION>
25%
</SOLUTION><end_of_turn>


# Let's try another example directly from our FinQA dataset

In [29]:
# Get a random example from our dev dataset
import random
random_idx = random.randint(0, len(dev_dataset)-1)
test_question = dev_dataset[random_idx]["prompt"][1]["content"]
ground_truth = dev_dataset[random_idx]["answer"]

print(f"Question: {test_question}...")
print(f"Ground truth answer: {ground_truth}")

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": test_question},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    tokenize = False,
)
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 400,
    temperature = 0.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Question: the following graph compares the cumulative 5-year total return to shareholders of cadence design systems , inc . 2019s common stock relative to the cumulative total returns of the s & p 500 index , the nasdaq composite index and the s & p information technology index . the graph assumes that the value of the investment in the company 2019s common stock and in each of the indexes ( including reinvestment of dividends ) was $ 100 on december 29 , 2001 and tracks it through december 30 , 2006 . comparison of 5 year cumulative total return* among cadence design systems , inc. , the s & p 500 index , the nasdaq composite index and the s & p information technology index 12/30/0612/31/051/1/051/3/0412/28/0212/29/01 cadence design systems , inc . nasdaq composite s & p information technology s & p 500 * $ 100 invested on 12/29/01 in stock or on 12/31/01 in index-incuding reinvestment of dividends . indexes calculated on month-end basis . copyright b7 2007 , standard & poor 2019s , a

In [30]:
# Get a random example from our dev dataset
import random
random_idx = random.randint(0, len(dev_dataset)-1)
test_question = dev_dataset[random_idx]["prompt"][1]["content"]
ground_truth = dev_dataset[random_idx]["answer"]

print(f"Question: {test_question}...")
print(f"Ground truth answer: {ground_truth}")

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": test_question},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    tokenize = False,
)
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 400,
    temperature = 0.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Question: the following table summarizes our future estimated cash payments under existing contractual obligations , including payments due by period: .

in millions | payments due by fiscal year total | payments due by fiscal year 2020 | payments due by fiscal year 2021 -22 | payments due by fiscal year 2023 -24 | payments due by fiscal year 2025 and thereafter
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
long-term debt ( a ) | $ 13093.0 | $ 1396.3 | $ 3338.4 | $ 2810.2 | $ 5548.1
accrued interest | 92.6 | 92.6 | - | - | -
operating leases ( b ) | 482.6 | 120.0 | 186.7 | 112.9 | 63.0
capital leases | 0.3 | 0.2 | 0.1 | - | -
purchase obligations ( c ) | 2961.8 | 2605.1 | 321.9 | 27.6 | 7.2
total contractual obligations | 16630.3 | 4214.2 | 3847.1 | 2950.7 | 5618.3
other long-term obligations ( d ) | 1302.4 | - | - | - | -


# Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's push_to_hub for an online save or save_pretrained for a local save.
[NOTE] This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [28]:
model.save_pretrained("/content/drive/MyDrive/2025_ConvFinQA_SFT_Agentic/gemma-3-4b-finqa-GRPO")  # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/2025_ConvFinQA_SFT_Agentic/gemma-3-4b-finqa-GRPO")
# model.push_to_hub("HF_ACCOUNT/gemma-3-1b-finqa", token = "...") # Online saving
# tokenizer.push_to_hub("HF_ACCOUNT/gemma-3-1b-finqa", token = "...") # Online saving

['/content/drive/MyDrive/2025_ConvFinQA_SFT_Agentic/gemma-3-4b-finqa-GRPO/processor_config.json']